In [10]:
"""
https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat/
AssistantAgent 設計為充當 AI 助手，預設使用LLMs但不需要人工輸入或代碼執行。它可以編寫 Python 代碼（在 Python 編碼塊中），供使用者在收到消息（通常是需要解決的任務的描述）時執行。在後台，Python 代碼是由LLM（例如 GPT-4）編寫的。它還可以接收執行結果並提出更正或錯誤修復建議。可以通過傳遞新的系統消息來更改其行為。LLM推理配置可以通過 [ llm_config ] 進行配置。
UserProxyAgent 人類的代理，默認情況下，在每次交互時徵求人類輸入作為代理的回復，並且還具有執行代碼和調用函數或工具的能力。當它在接收到的消息中檢測到可執行代碼塊並且沒有提供人工使用者輸入時，它會自動 UserProxyAgent 觸發代碼執行。可以通過將 code_execution_config 參數設置為 False 來禁用代碼執行。
默認情況下不是基於LLM的回覆(所以是誰回覆的???)，LLM-based response is disabled by default.。可以通過設置為 llm_config 與推理配置相對應的字典來啟用它。When llm_config 設置為字典，則當 UserProxyAgent 未執行代碼，就還是可以使用 LLM 來生成回復。

＊＊＊＊＊通過下列程式的執行過程，autogen 調用 gpt3.5 時，連調用函式的程式碼似乎也都會自己生成(也就是靠自然語言就可讓gpt3.5產生要調用函式的程式碼，厲害)
"""
import autogen
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-1106"], # gpt-3.5-turbo-16k, gpt-3.5-turbo, gpt-3.5-turbo-1106
    },
)
# 下面是另一種方式
llm_config={"config_list": config_list, "seed": 42}

coder = autogen.AssistantAgent("coder", llm_config=llm_config, max_consecutive_auto_reply=2)

user_proxy = autogen.UserProxyAgent("user_proxy", 
                                    llm_config=llm_config, 
                                    human_input_mode="NEVER", # user_proxy 人工介入模式：NONE, NEVER, TERMINATE, ALWAYS 四種選項, 差異???
                                    max_consecutive_auto_reply=2,
                                    code_execution_config={"work_dir": "coding", "use_docker": False})

task = "Plot a chart of 'META' and 'MSFT' stock price current YTD. Save the result to a file named nvda_tesla.png"
user_proxy.initiate_chat(coder, message = task)

user_proxy (to coder):

Plot a chart of 'META' and 'MSFT' stock price current YTD. Save the result to a file named nvda_tesla.png

--------------------------------------------------------------------------------
coder (to user_proxy):

To plot a chart of 'META' and 'MSFT' stock price current YTD, we can use the yfinance library to fetch the stock price data and the matplotlib library to plot the chart. First, we'll need to install the yfinance library if it's not already installed. Then, we can fetch the stock price data and plot the chart.

Here's the Python code to do this:

```python
# filename: stock_price_chart.py
import yfinance as yf
import matplotlib.pyplot as plt

# Fetch the stock price data for 'META' and 'MSFT'
meta_data = yf.download('META', start='2022-01-01')
msft_data = yf.download('MSFT', start='2022-01-01')

# Plot the stock price data
plt.figure(figsize=(12, 6))
plt.plot(meta_data['Close'], label='META')
plt.plot(msft_data['Close'], label='MSFT')
plt.title('META and 

ChatResult(chat_history=[{'content': "Plot a chart of 'META' and 'MSFT' stock price current YTD. Save the result to a file named nvda_tesla.png", 'role': 'assistant'}, {'content': "To plot a chart of 'META' and 'MSFT' stock price current YTD, we can use the yfinance library to fetch the stock price data and the matplotlib library to plot the chart. First, we'll need to install the yfinance library if it's not already installed. Then, we can fetch the stock price data and plot the chart.\n\nHere's the Python code to do this:\n\n```python\n# filename: stock_price_chart.py\nimport yfinance as yf\nimport matplotlib.pyplot as plt\n\n# Fetch the stock price data for 'META' and 'MSFT'\nmeta_data = yf.download('META', start='2022-01-01')\nmsft_data = yf.download('MSFT', start='2022-01-01')\n\n# Plot the stock price data\nplt.figure(figsize=(12, 6))\nplt.plot(meta_data['Close'], label='META')\nplt.plot(msft_data['Close'], label='MSFT')\nplt.title('META and MSFT Stock Price YTD')\nplt.xlabel('Da

In [9]:
import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent
"""
20240219，此處列表更新為5類
gpt35、gpt4、gpt4v、gemini、gemini_vision
"""
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-16k-0613"],
    },
)
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0613", "gpt-4-0314", "gpt-4-1106-preview"],
    },
)
config_list_gpt4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-vision-preview", "dalle"],
    },
)
config_list_gemini = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro"],
    },
)
config_list_gemini_vision = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro-vision"],
    },
)

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-1106"], # gpt-3.5-turbo-16k, gpt-3.5-turbo, gpt-3.5-turbo-1106
    },
)
llm_config={"config_list": config_list, "seed": 42}

# 定義2個Agent，assistant和user_proxy
# 但不清楚為何跟下面區塊直接設定使用哪個模型不一樣，這裡用 "assistant" 跟 "user_proxy"
# 下面的區塊是用 "gpt-3.5-turbo-1106"、"gemini-pro",
assistant = AssistantAgent(
    "assistant", llm_config={"config_list": config_list_gpt35, "seed": 42}, max_consecutive_auto_reply=3
)
# print(assistant.system_message)
# 這個 user_proxy 模型用 "user_proxy" ??? 不懂
# 也沒有 llm_config，但還是可以執行???
# UserProxyAgent 應該只能使用 openAI，其他會報錯，但這個區塊卻又可以????? 不過，整個來說，幾乎都是不行的。
user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={"work_dir": "coding", "use_docker": False},
    human_input_mode="NEVER",
    llm_config={"config_list": config_list_gpt35, "seed": 42},
    is_termination_msg=lambda x: content_str(x.get("content")).find("TERMINATE") >= 0,
)
# 由 user_proxy開始發起對話(initiate_chat)
user_proxy.initiate_chat(assistant, message="請搜尋arXiv上有關醫學領域蛋白質研究最前緣的技術有哪些")  # 這個會報錯
# user_proxy.initiate_chat(assistant, message="Sort the array with Bubble Sort: [4, 1, 5, 2]")     # 這個卻可以 ????


TypeError: OpenAI.__init__() got an unexpected keyword argument 'api_version'

In [ ]:
# 增加一個 agent
# define a custom agent
pm = autogen.AssistantAgent("Product_manager", system_message="Creative in software product ideas.", llm_config=llm_config)

groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[])
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config, human_input_mode="NEVER")

user_proxy = autogen.UserProxyAgent("user_proxy", 
                                    human_input_mode="NEVER",
                                    code_execution_config={"work_dir": "coding", "use_docker": False})
coder = autogen.AssistantAgent("coder", llm_config=llm_config)

# task = "Find a lastest paper about gpt-4 on arxiv and and find its potential applications in software. Final result should reply in traditional chinese."
task = "Find a lastest paper about gpt-4 on arxiv and and find its potential applications in software."
user_proxy.initiate_chat(manager, message = task)

In [ ]:
import autogen
from autogen import AssistantAgent, UserProxyAgent
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-1106"],
    },
)
llm_config={"config_list": config_list, "seed": 42}

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent("user_proxy", 
                                    human_input_mode="NEVER",
                                    code_execution_config={"work_dir": "coding", "use_docker": False})

# create an AssistantAgent instance named "assistant"｝不指定 llm_config 會出現警告訊息
assistant = AssistantAgent(name="assistant", llm_config=llm_config)